In [ ]:
import pandas as pd
df = pd.read_csv("train.csv")

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import os

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c elo-merchant-category-recommendation

In [ ]:
import zipfile

with zipfile.ZipFile("elo-merchant-category-recommendation.zip", 'r') as zip_ref:
    #zip_ref.extractall("talkingdata_adtracking")

!ls talkingdata_adtracking

Start from here we run the code of the ADS.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)

**3.1 Data Loading and Memory Optimization**

First, we load the Elo Merchant Category Recommendation dataset and apply a memory reduction function to optimize memory usage. The function reduce_mem_usage downcasts numeric columns to more efficient types where possible. This helps speed up processing and avoids running out of memory when dealing with the large transaction tables.

In [ ]:
!pip install pandas numpy lightgbm xgboost catboost fairlearn shap

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        if col_type in [np.object_, object]:
            # Skip object types (e.g., strings) for downcasting
            continue
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            # Downcast integers to smallest possible subdtype
            if str(col_type)[:3] == 'int':
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                # (If data fits in int64, we leave as is)
            # Downcast floats to float32/16 if possible without losing precision
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                # (If outside float32 range, keep as float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f'Memory usage decreased to {end_mem:5.2f} MB '
              f'({100 * (start_mem - end_mem) / start_mem:.1f}% reduction)')
    return df

In the code above, after downloading and extracting the dataset, we define reduce_mem_usage to convert numeric columns to smaller types where possible. This function will be applied to the transaction data tables to significantly cut down memory usage (often a reduction of 25-50% or more).

**3.2 Data Cleaning and Basic Feature Engineering**

Next, we load the transaction datasets and perform data cleaning steps such as parsing dates, handling missing values, and converting categorical flags to binary. We also create initial features like the time difference between first activation and a reference date.

In [ ]:
new_transactions = pd.read_csv('talkingdata_adtracking/new_merchant_transactions.csv',
                               parse_dates=['purchase_date'])

historical_transactions = pd.read_csv('talkingdata_adtracking/historical_transactions.csv',
                                      parse_dates=['purchase_date'])

# Convert boolean-like flags from 'Y'/'N' to 1/0
def binarize(df):
    for col in ['authorized_flag', 'category_1']:
        if col in df.columns:
            df[col] = df[col].map({'Y': 1, 'N': 0})
    return df

historical_transactions = binarize(historical_transactions)
new_transactions = binarize(new_transactions)


train = pd.read_csv('train.csv', parse_dates=['first_active_month'])
test = pd.read_csv('test.csv', parse_dates=['first_active_month'])
train['elapsed_time'] = (pd.to_datetime('2018-02-01') - train['first_active_month']).dt.days
test['elapsed_time'] = (pd.to_datetime('2018-02-01') - test['first_active_month']).dt.days

# Separate target from train data
target = train['target']
#train.drop('target', axis=1, inplace=True)


We load historical_transactions and new_transactions, which contain transaction history for each card (card_id). The purchase_date column is parsed as datetime.
We define a helper binarize to map categorical flags authorized_flag (which indicates if a historical transaction was approved) and category_1 from 'Y'/'N' to 1/0. This makes them numeric binary features.

We load train.csv and test.csv, which contain one row per card with some basic features (feature_1, feature_2, feature_3 are anonymized card attributes, and first_active_month when the card was first used). We parse first_active_month as a date.


We create an elapsed_time feature measuring the number of days between the card's first active month and a reference date (Feb 1, 2018, which is around the dataset cutoff). This captures card age (older cards will have larger elapsed_time).

At this point, basic data cleaning is done: missing values in first_active_month (if any) would result in elapsed_time being NaN which we could fill with a default (though in this dataset there were no missing first_active_month in train/test). The flag conversions handle boolean fields. We will next engineer more features from the transaction history.

**Feature engineering**

The ADS added the following features followed the result of [this kernel](https://www.kaggle.com/code/chauhuynh/my-first-kernel-3-699)

**3.3 Feature Engineering from Transaction History**

We will aggregate the transaction data to create features at the card level. This involves generating summary statistics from both the historical transactions (including authorized and unauthorized transactions) and the new merchant transactions.

Steps include:


Date-based features: We compute how long ago each transaction happened relative to a reference (today or a fixed date) and combine with the provided month_lag (the month index relative to the reference date in the dataset).
One-hot encoding categorical features: The transaction tables have categorical fields category_2 and category_3. We create one-hot encoded dummy variables for these to use in aggregation.


Aggregations: For each card, we calculate aggregated metrics (sum, mean, std, etc.) of transaction amounts, installment counts, and other fields, both overall and grouped by certain categories (like by month lag, or by whether the transaction was authorized).


Separating authorized transactions: We split historical transactions into authorized and unauthorized sets to derive separate features from each, as their patterns might influence loyalty differently.


Successive transaction aggregates: We create some advanced features capturing the relationship between two fields (e.g., how purchase amounts vary by category or city for each card).


In [ ]:
import datetime

# Create 'month_diff' feature: how many months since the purchase
historical_transactions['month_diff'] = ((datetime.datetime.today() - historical_transactions['purchase_date']).dt.days // 30)
historical_transactions['month_diff'] += historical_transactions['month_lag']

new_transactions['month_diff'] = ((datetime.datetime.today() - new_transactions['purchase_date']).dt.days // 30)
new_transactions['month_diff'] += new_transactions['month_lag']

historical_transactions = pd.get_dummies(historical_transactions, columns=['category_2', 'category_3'])
new_transactions = pd.get_dummies(new_transactions, columns=['category_2', 'category_3'])

# Reduce memory usage
historical_transactions = reduce_mem_usage(historical_transactions, verbose=True)
new_transactions = reduce_mem_usage(new_transactions, verbose=True)

# Aggregation: authorized_flag mean per card (what fraction of transactions were authorized)
agg_auth = historical_transactions.groupby('card_id')['authorized_flag'].mean().reset_index()
agg_auth.columns = ['card_id', 'authorized_flag_mean']  # fraction of transactions authorized for each card

# Split historical transactions
authorized_transactions = historical_transactions[historical_transactions['authorized_flag'] == 1]
unauthorized_transactions = historical_transactions[historical_transactions['authorized_flag'] == 0]

# Add a purchase month feature (month of year) for potential seasonality
for df in [authorized_transactions, unauthorized_transactions, new_transactions]:
    df['purchase_month'] = df['purchase_date'].dt.month


We compute month_diff for both historical and new transactions, which estimates how many months ago the transaction occurred. It uses purchase_date and current date (as a stand-in for a consistent reference) and adjusts by month_lag (a feature given in the dataset indicating how far back each historical transaction is relative to the reference month for that card). This feature can capture recency of transactions.


We apply one-hot encoding to category_2 and category_3 in the transaction data. These were categorical variables with a small number of possible values, now turned into dummy binary columns (e.g., category_2_1.0, category_2_2.0, ..., category_3_A, category_3_B, etc.). This allows us to compute meaningful aggregates like the percentage of transactions of each category for a card.
We use reduce_mem_usage on the expanded transaction tables, since one-hot encoding can increase the number of columns significantly.


We then derive authorized_flag_mean for each card as the mean of the authorized_flag column in historical transactions. This gives the proportion of transactions that were approved (Y) for each card. This could be indicative of card usage stability or credit issues (cards with lower authorization rates might be experiencing more declines).


We separate the historical_transactions into two DataFrames: one for authorized transactions and one for unauthorized (declined) transactions. We will aggregate features from these separately to see if they have different predictive power.


We add a purchase_month feature to capture the month of year when the purchase happened, which could detect seasonal usage patterns (for example, higher spending in December holidays, etc.).


Next, we define an aggregation function to compute a variety of features for each card from a given transaction table. We will apply this to the authorized transactions, unauthorized (historical) transactions, and new transactions.

In [ ]:
# Define an aggregation function for transaction history
def aggregate_transactions(history_df):
    history_df.loc[:, 'purchase_date'] = pd.DatetimeIndex(history_df['purchase_date']).astype(np.int64) * 1e-9
    agg_func = {
        'authorized_flag': ['sum', 'mean'],
        'category_1': ['sum', 'mean'],
        **{col: ['mean'] for col in history_df.columns if col.startswith('category_2_') or col.startswith('category_3_')},
        'merchant_id': ['nunique'],
        'merchant_category_id': ['nunique'],
        'state_id': ['nunique'],
        'city_id': ['nunique'],
        'subsector_id': ['nunique'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_month': ['mean', 'max', 'min', 'std'],
        'purchase_date': [np.ptp, 'min', 'max'],
        'month_lag': ['mean', 'max', 'min', 'std'],
        'month_diff': ['mean']
    }
    # Perform groupby aggregation by card_id
    agg_df = history_df.groupby('card_id').agg(agg_func)
    # Flatten multi-level column index resulting from aggregation
    agg_df.columns = ['_'.join(col).strip() for col in agg_df.columns.values]
    agg_df.reset_index(inplace=True)
    # Also count the number of transactions per card
    count_df = history_df.groupby('card_id').size().reset_index(name='transactions_count')
    # Merge count into agg_df
    agg_df = agg_df.merge(count_df, on='card_id', how='left')
    return agg_df

# Aggregate features for each subset of transactions
hist_agg = aggregate_transactions(unauthorized_transactions)
auth_agg = aggregate_transactions(authorized_transactions)
new_agg = aggregate_transactions(new_transactions)

hist_agg.columns = ['hist_' + c if c != 'card_id' else c for c in hist_agg.columns]
auth_agg.columns = ['auth_' + c if c != 'card_id' else c for c in auth_agg.columns]
new_agg.columns = ['new_' + c if c != 'card_id' else c for c in new_agg.columns]

print(hist_agg.head(3))
print(f"Unauthorized transaction features: {hist_agg.shape[1]-1} columns (excluding card_id)")
print(auth_agg.head(3))
print(f"Authorized transaction features: {auth_agg.shape[1]-1} columns")
print(new_agg.head(3))
print(f"New transaction features: {new_agg.shape[1]-1} columns")


In the aggregation function above, we specify a dictionary agg_func to calculate numerous features for each card:


- For binary flags like authorized_flag (in the context of the full historical data) and category_1, we take sum (count of True occurrences) and mean (percentage of transactions that are True).


- For each one-hot encoded category column (those starting with category_2_ or category_3_), we take the mean, which effectively gives the fraction of transactions of that category for the card.


- We compute the number of unique values of merchant-related categorical fields (merchant_id, merchant_category_id, state_id, city_id, subsector_id) to capture the diversity of the card’s transactions.


- For continuous/numeric fields like purchase_amount and installments, we gather summary statistics: total sum, mean, max, min, and standard deviation per card.
For purchase_month, we get mean, max, min, std to see spending patterns across months of the year.


- For purchase_date (converted to a numeric timestamp in seconds), we compute the range (ptp which is max-min), as well as the min and max purchase timestamp for each card (which could indicate the first and last transaction times).

- For month_lag and month_diff, we get mean and other stats (for month_lag) to understand recency/spread of transactions.


After aggregation, we merge in the total count of transactions (transactions_count) for each card. We then do this for unauthorized historical transactions (hist_agg), authorized transactions (auth_agg), and new merchant transactions (new_agg). We add prefixes to each column name (hist_, auth_, new_) so that features from different sources are distinct when we merge them together. The quick print statements show the first few rows and number of features generated from each source to confirm our aggregation worked as expected.

**3.4 Additional Feature Engineering (Successive Aggregates)**

We introduce a set of “successive aggregation” features from the new transactions data to capture interactions between different fields. For example, how the purchase amount varies with certain categorical fields for each card. The idea is to group by card and one field, then aggregate another field within that grouping, and then aggregate across the groups for each card.

In [ ]:
def successive_aggregates(df, field1, field2):
    temp = df.groupby(['card_id', field1])[field2].mean()
    temp_df = pd.DataFrame(temp).reset_index()
    agg_temp = temp_df.groupby('card_id')[field2].agg(['mean', 'min', 'max', 'std']).reset_index()
    agg_temp.columns = [field1 + '_' + field2 + '_' + col for col in agg_temp.columns.values]
    agg_temp = agg_temp.rename(columns={agg_temp.columns[0]: 'card_id'})
    return agg_temp

# Apply successive aggregations on new_transactions for various combinations
add_feat1 = successive_aggregates(new_transactions, 'category_1', 'purchase_amount')
add_feat2 = successive_aggregates(new_transactions, 'installments', 'purchase_amount')
add_feat3 = successive_aggregates(new_transactions, 'city_id', 'purchase_amount')
add_feat4 = successive_aggregates(new_transactions, 'category_1', 'installments')

# Merge these additional features on card_id
additional_fields = add_feat1.merge(add_feat2, on='card_id', how='left')
additional_fields = additional_fields.merge(add_feat3, on='card_id', how='left')
additional_fields = additional_fields.merge(add_feat4, on='card_id', how='left')

print(additional_fields.head(3))
print(f"Additional interaction features from new transactions: {additional_fields.shape[1]-1} columns")


Here we define successive_aggregates to capture second-order interactions:


- For example, successive_aggregates(new_transactions, 'category_1', 'purchase_amount'): for each card, it finds the average purchase amount for each value of category_1 (which is binary 0/1 in this context, representing some category flag in the card’s profile), then for that card it computes the mean, min, max, std of those two average values. Essentially, this yields features that describe differences in spending when category_1 is 0 vs 1 for a card.


We do similar for 'installments' vs purchase amount (how purchase amount varies by number of installments), 'city_id' vs purchase amount (how spending differs across cities for the card), and 'category_1' vs 'installments' (how installment counts differ by category_1).


We then merge all these new interaction features into one DataFrame additional_fields keyed by card_id. These features might help capture behavior nuances in the new merchant transactions that aren’t directly captured by simple aggregates.

**3.5 Merging All Features**

Now that we have engineered numerous features from different data sources, we merge them into the training and testing datasets (by card_id). After merging, we perform any final feature selection or reduction, such as dropping highly correlated features that might not add value, to avoid multicollinearity.

In [ ]:
# Merge all feature tables into train and test dataframes
# At this point, train and test contain the card-level base features (feature_1, feature_2, feature_3, first_active_month, elapsed_time)
train = train.merge(hist_agg, on='card_id', how='left')
test = test.merge(hist_agg, on='card_id', how='left')

train = train.merge(auth_agg, on='card_id', how='left')
test = test.merge(auth_agg, on='card_id', how='left')

train = train.merge(new_agg, on='card_id', how='left')
test = test.merge(new_agg, on='card_id', how='left')

train = train.merge(agg_auth, on='card_id', how='left')
test = test.merge(agg_auth, on='card_id', how='left')

train = train.merge(additional_fields, on='card_id', how='left')
test = test.merge(additional_fields, on='card_id', how='left')

print(f"Train shape after merging: {train.shape}, Test shape: {test.shape}")


After merging, the train and test DataFrames now contain all the features we created:

- Base card features (feature_1, feature_2, feature_3, elapsed_time, etc.).

- Aggregated features from unauthorized historical transactions (prefixed hist_).

- Aggregated features from authorized historical transactions (prefixed auth_).

- Aggregated features from new merchant transactions (prefixed new_).

- The fraction of authorized transactions (authorized_flag_mean).

- Additional interaction features from new transactions (like category_1_purchase_amount_mean, etc., from additional_fields).

If any features are missing for a card, they will remain NaN after the merge. We should fill these missing values with 0 or another appropriate value for modeling, since a lack of transactions can be represented as zeros in the aggregate features.

In [ ]:
def clean_feature_names(df):
    df.columns = [re.sub(r"[^A-Za-z0-9_]+", "_", col) for col in df.columns]
    return df

train = clean_feature_names(train)
test = clean_feature_names(test)

#features = [col for col in train.columns if col not in ['card_id', 'first_active_month']]
features = [col for col in train.columns if col not in ['card_id', 'first_active_month', 'target']]
categorical_feats = ['feature_2', 'feature_3']

train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

We replace all NaNs with 0, assuming that absence of data can be treated as no contribution. Now, features contains the list of all feature column names we will use (excluding non-predictive ID or date columns). We also specify categorical_feats for LightGBM – in this dataset, feature_2 and feature_3 are known to be categorical codes (even though they are numeric values, they represent categories).

**Feature Reduction:**

In [ ]:
# Identify and remove highly correlated features
corr_matrix = train[features].corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

to_drop = [col for col in upper_tri.columns if any(upper_tri[col] > 0.98)]
print(f"Highly correlated features (corr > 0.98) to drop: {to_drop}")

features = [f for f in features if f not in to_drop]

The code above computes absolute correlations between all feature pairs and finds columns that have any correlation above 0.98 with another. Those identified in to_drop are removed from our feature list.

**3.6 Model Training with LightGBM (5-Fold Cross-Validation)**

Now we train a LightGBM model to predict the loyalty score. We use 5-fold cross-validation for reliable estimation of performance, with early stopping in each fold to prevent overfitting. We will also record feature importance from each fold.

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

# LightGBM
params = {
    'num_leaves': 111,
    'min_data_in_leaf': 149,
    'objective': 'regression',
    'max_depth': 9,
    'learning_rate': 0.005,
    'boosting': 'gbdt',
    'feature_fraction': 0.7522,
    'bagging_fraction': 0.7083,
    'bagging_seed': 11,
    'metric': 'rmse',
    'lambda_l1': 0.2634,
    'random_state': 133,
    'verbosity': -1
}

folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof_preds = np.zeros(len(train))
test_preds = np.zeros(len(test))
feature_importance_df = pd.DataFrame()

print("Starting 5-fold cross-validation for LightGBM...")
for fold, (train_idx, val_idx) in enumerate(folds.split(train, target)):
    print(f"Fold {fold+1}")

    train_data = lgb.Dataset(train.iloc[train_idx][features],
                             label=target.iloc[train_idx],
                             categorical_feature=categorical_feats,
                             free_raw_data=False)

    valid_data = lgb.Dataset(train.iloc[val_idx][features],
                             label=target.iloc[val_idx],
                             categorical_feature=categorical_feats,
                             free_raw_data=False)

    clf = lgb.train(
        params,
        train_data,
        num_boost_round=10000,
        valid_sets=[train_data, valid_data],
        valid_names=['train', 'valid'],
        callbacks=[
            lgb.early_stopping(200),
            lgb.log_evaluation(period=100)
        ]
    )

    oof_preds[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    test_fold_pred = clf.predict(test[features], num_iteration=clf.best_iteration)
    test_preds += test_fold_pred / folds.n_splits

    fold_importance = pd.DataFrame({
        'feature': features,
        'importance': clf.feature_importance(),
        'fold': fold+1
    })
    feature_importance_df = pd.concat([feature_importance_df, fold_importance], axis=0)

    print(f"Fold {fold+1} RMSE: {mean_squared_error(target.iloc[val_idx], oof_preds[val_idx])**0.5:.4f} "
          f"(best iteration: {clf.best_iteration})")

cv_rmse = mean_squared_error(target, oof_preds)**0.5
print(f"\n5-fold CV RMSE: {cv_rmse:.4f}")

We set up LightGBM parameters. The objective is regression (predicting a continuous loyalty score). We use a fairly low learning rate (0.005) and allow up to 10000 boosting rounds with early stopping after 200 rounds of no improvement.

We perform 5-fold cross-validation using KFold. For each fold, we create a training set and validation set. We use lgb.Dataset to prepare data, specifying which features are categorical so LightGBM can use them appropriately (with handling of categorical splits).

We train the model with lgb.train, including the validation set to monitor RMSE. Early stopping is used to halt training when the validation score stops improving for 200 rounds, which helps prevent overfitting.

We store out-of-fold predictions in oof_preds for the validation part of each fold, and we accumulate test set predictions by averaging the predictions from each fold's model.

We collect feature importances from each fold into feature_importance_df.
After all folds, we calculate the overall cross-validated RMSE on the training data. This gives us an estimate of model performance on unseen data.


**3.7 Feature Importance and Validation Results**

visualize the feature importance to see which features the model found most useful. We will average the importance across the folds and plot the top 20 features. We also take a look at the distribution of predictions for the test set to ensure they are in a reasonable range and not wildly extrapolating beyond the training targets.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

avg_importance = feature_importance_df.groupby('feature')['importance'].mean().reset_index()
avg_importance.sort_values(by='importance', ascending=False, inplace=True)
top_feats = avg_importance.head(20)

# Plot top 20 feature importances
plt.figure(figsize=(8, 10))
sns.barplot(x='importance', y='feature', data=top_feats, orient='h')
plt.title('Top 20 Feature Importances (LightGBM - avg over folds)')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

plt.figure(figsize=(8,5))
sns.histplot(test_preds, bins=50, kde=True)
plt.title('Distribution of Test Set Predictions')
plt.xlabel('Predicted Loyalty Score')
plt.grid(True)
plt.show()

# Summary stats of predictions
pred_series = pd.Series(test_preds, name="Predictions")
print(pred_series.describe())

We output a bar chart of the top 20 features by importance. Typically, we expect features derived from purchase amounts and transaction counts to be among the top. For example, one might see features like hist_purchase_amount_sum (total historical spend), new_purchase_amount_sum (total new spend), or hist_month_diff_mean (average recency of historical transactions) as highly important. Categorical card features like feature_2 or feature_3 might also appear if they correlate with loyalty. The distribution of test predictions plot shows how the predicted loyalty scores are spread for the test set. We print summary statistics of the predictions to check for any extreme values.

**3.8 Hyperparameter Tuning with Optuna**

In this section, we perform hyperparameter tuning on the LightGBM model using Optuna. We define a search space for key parameters and use 5-fold cross-validation to evaluate the performance (minimizing RMSE in this case) of each trial. The best parameters found can then be used to retrain the model for improved performance.

In [ ]:
## THIS IS GOOD BUT IT TAKES FOREVER


import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
import numpy as np

X = train[features]
y = train['target']

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 150),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'random_state': 42
    }

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = []

    for train_idx, val_idx in kf.split(X):
        train_data = lgb.Dataset(X.iloc[train_idx], label=y.iloc[train_idx])
        valid_data = lgb.Dataset(X.iloc[val_idx], label=y.iloc[val_idx])

        model = lgb.train(
            params,
            train_data,
            valid_sets=[valid_data],
            num_boost_round=10000,
            callbacks=[
                lgb.early_stopping(200),
                lgb.log_evaluation(period=100)
            ]
        )

        preds = model.predict(X.iloc[val_idx], num_iteration=model.best_iteration)
        rmse = np.sqrt(mean_squared_error(y.iloc[val_idx], preds))
        rmse_scores.append(rmse)

    return np.mean(rmse_scores)

# Run Optuna Study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print("Best Params Found by Optuna:", study.best_params)
print("Best CV RMSE:", study.best_value)


In [ ]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np

X = train[features]
y = train['target']

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 5, 12),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
        'random_state': 42
    }

    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    rmse_scores = []

    for train_idx, val_idx in kf.split(X):
        train_data = lgb.Dataset(X.iloc[train_idx], label=y.iloc[train_idx])
        valid_data = lgb.Dataset(X.iloc[val_idx], label=y.iloc[val_idx])

        model = lgb.train(
            params,
            train_data,
            valid_sets=[valid_data],
            num_boost_round=3000,
            callbacks=[
                lgb.early_stopping(100),
                lgb.log_evaluation(period=100)
            ]
        )

        preds = model.predict(X.iloc[val_idx], num_iteration=model.best_iteration)
        rmse = np.sqrt(mean_squared_error(y.iloc[val_idx], preds))
        rmse_scores.append(rmse)

    return np.mean(rmse_scores)

# Run Optuna Study quicklyTT
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20, n_jobs=-1)

print("✅ Best Params Found by Optuna (Quick Version):", study.best_params)
print("📊 Best CV RMSE (Quick Version):", study.best_value)


The printed best parameters and RMSE indicate the optimal setting found by Optuna.

**3.9 Evaluation Metrics (RMSE, MAE, QWK)**

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, cohen_kappa_score
import numpy as np

oof_predictions = oof_preds
true_values = target

# 1️RMSE
rmse = np.sqrt(mean_squared_error(true_values, oof_predictions))
print(f"RMSE (Root Mean Squared Error): {rmse:.4f}")

# 2️MAE
mae = mean_absolute_error(true_values, oof_predictions)
print(" MAE (Mean Absolute Error): {mae:.4f}")

# 3️Quadratic Weighted Kappa (QWK)
def discretize_preds(preds, bins=5):
    return np.digitize(preds, np.linspace(min(preds), max(preds), bins))

pred_classes = discretize_preds(oof_predictions, bins=5)
true_classes = discretize_preds(true_values, bins=5)

qwk = cohen_kappa_score(true_classes, pred_classes, weights='quadratic')
print(f" QWK (Quadratic Weighted Kappa): {qwk:.4f}")


Lower RMSE/MAE indicates better predictive accuracy (error closer to 0). QWK closer to 1 means the model's predictions have strong agreement with true labels on an ordinal scale. These metrics help validate model consistency across folds and overall.

**Part 4: Outcomes**

Defining Subgroups for Analysis

To assess our model's accuracy and fairness across different subpopulations, we define three subgroup attributes:

- Spending tier: Divide feature_1 (an indicator of cardholder purchasing power) into quintiles (5 tiers, from Tier1 = lowest spend to Tier5 = highest spend). This groups cardholders by overall spending level.

- Recency: Categorize hist_month_lag_mean (average transaction recency) into three buckets: 'older' for values ≤ -2 (transactions mostly in the distant past), 'recent' for value = -1, and 'current' for values ≥ 0 (mostly recent transactions). This captures how recent or stale each card's transaction history is.

- Authorization status: Classify authorized_flag_mean (fraction of transactions approved) into HighAuth (≥ 0.5) and LowAuth (< 0.5). HighAuth cards had a majority of transactions authorized, whereas LowAuth cards saw more declines, indicating different usage behaviors.


**Prepare train_results DataFrame with Predictions and Subgroup Labels**

We will create a DataFrame train_results that contains each training example’s actual target (loyalty_score), the model’s predicted score (our out-of-fold predictions), and the subgroup labels defined above.

In [ ]:
train_results = pd.DataFrame({
    'card_id': train['card_id'],
    'actual': target,
    'prediction': oof_preds
})

In [ ]:
if historical_transactions['authorized_flag'].dtype == object:
    historical_transactions['authorized_flag'] = historical_transactions['authorized_flag'].map({'Y': 1, 'N': 0}).astype(np.int8)

# month_diff
import datetime
historical_transactions['month_diff'] = (
    (datetime.datetime.today() - historical_transactions['purchase_date']).dt.days // 30
    + historical_transactions['month_lag']
)

# agg
agg = (
    historical_transactions
    .groupby('card_id', as_index=False)
    .agg({
        'month_diff': 'mean',
        'authorized_flag': 'mean'
    })
    .rename(columns={'month_diff': 'hist_month_diff_mean', 'authorized_flag': 'authorized_flag_mean'})
)

train_results = train_results.merge(agg, on='card_id', how='left')

print(train_results[['hist_month_diff_mean', 'authorized_flag_mean']].head())


In [ ]:
print(train_results.columns)
print(train.columns)

In [ ]:
# 1. Spending tier (quintiles of hist_month_diff_mean)
tmp = pd.qcut(train_results['hist_month_diff_mean'], q=5, duplicates='drop')
labels = [f"Tier{i}" for i in range(1, tmp.cat.categories.size + 1)]
train_results['spending_tier'] = pd.qcut(
    train_results['hist_month_diff_mean'], q=5, labels=labels, duplicates='drop'
)
# 2. Recency from hist_month_diff_mean
rec_rounded = np.round(train_results['hist_month_diff_mean']).clip(-2, 0)
train_results['recency'] = rec_rounded.map({-2:'older', -1:'recent', 0:'current'})

# 3. Authorization status
train_results['auth_status'] = np.where(
    train_results['authorized_flag_mean'] >= 0.5, 'HighAuth', 'LowAuth'
)

In [ ]:
display(train_results[['spending_tier', 'recency', 'auth_status']].agg(['count', 'nunique']))

The code above adds three new columns to train_results. We use pd.qcut to create approximately equal-sized quintile groups for feature_1. We then map the recency and authorization features to descriptive categorical labels. Finally, we display the count of instances and number of unique categories in each subgroup to ensure the grouping is as expected.

**Accuracy Metrics by Subgroup**

Next, we evaluate model performance within each subgroup using two common accuracy metrics: Root Mean Squared Error (RMSE) and Mean Absolute Error (MAE).

Lower values indicate better predictions. We compute RMSE and MAE for each category of each subgroup, allowing us to compare errors across groups:

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred))

# Compute RMSE and MAE per subgroup category
for col in ['spending_tier', 'recency', 'auth_status']:
    metrics_by_group = train_results.groupby(col).apply(
        lambda df: pd.Series({
            'RMSE': rmse(df['actual'], df['prediction']),
            'MAE': mean_absolute_error(df['actual'], df['prediction'])
        })
    )
    print(f"\nPerformance metrics by {col}:")
    display(metrics_by_group)


**Bootstrapped Confidence Intervals**

To gauge the uncertainty in these error estimates, we can compute bootstrapped 95% confidence intervals for the RMSE and MAE in each subgroup.

In [ ]:
import math
def bootstrap_ci(y_true, y_pred, metric_func, n_bootstrap=300):
    """Compute bootstrap 95% CI for a given metric."""
    stats = []
    n = len(y_true)
    for i in range(n_bootstrap):
        idx = np.random.randint(0, n, n)
        stats.append(metric_func(y_true.iloc[idx], y_pred.iloc[idx]))
    # 2.5th and 97.5th percentiles for 95% CI
    return np.percentile(stats, [2.5, 97.5])

for col in ['spending_tier', 'recency', 'auth_status']:
    print(f"\n95% bootstrap CI for metrics by {col}:")
    for group_val, group_df in train_results.groupby(col):
        rmse_lower, rmse_upper = bootstrap_ci(group_df['actual'], group_df['prediction'], rmse)
        mae_lower, mae_upper = bootstrap_ci(group_df['actual'], group_df['prediction'], mean_absolute_error)
        print(f"  {col} = {group_val}:  RMSE 95% CI = [{rmse_lower:.3f}, {rmse_upper:.3f}],  MAE 95% CI = [{mae_lower:.3f}, {mae_upper:.3f}]")

**Test Set Prediction Distribution**

It's also informative to examine the distribution of the model's predictions on the test set. This helps verify that the predictions fall in a sensible range and to compare with training target distribution for any shifts. Below, we visualize the distribution of test_preds:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
plt.hist(test_preds, bins=50, color='steelblue', edgecolor='black')
plt.title("Distribution of Test Set Predictions")
plt.xlabel("Predicted loyalty score")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

**Model Explainability with SHAP**

This section uses SHAP (SHapley Additive exPlanations) to interpret the trained LightGBM model. We compute SHAP values to understand feature importance and contributions. First, a global interpretation is provided via a summary plot (showing which features have the highest impact on model output across all samples). Then, we provide a local interpretation for an individual prediction using a force plot (to see how each feature influenced that specific prediction).

In [ ]:
model_to_explain = clf  # Use the final trained LightGBM model

In [ ]:
import shap

# Use the final trained LightGBM model for SHAP analysis
model_to_explain = clf

explainer = shap.TreeExplainer(model_to_explain)
X_train_sample = train[features]

if len(X_train_sample) > 10000:
    X_train_sample = X_train_sample.sample(10000, random_state=42)  # Sample for faster computation

shap_values = explainer.shap_values(X_train_sample)

# Global interpretation: Feature Importance Bar Plot
shap.summary_plot(shap_values, X_train_sample, plot_type="bar")

# Detailed global interpretation: SHAP Summary Plot
shap.summary_plot(shap_values, X_train_sample)

# Local interpretation: Explain an individual prediction
shap.initjs()
sample_index = 0  # Change this to select different samples
sample_features = X_train_sample.iloc[sample_index]
sample_shap_values = shap_values[sample_index]

shap.force_plot(explainer.expected_value, sample_shap_values, sample_features)

In the SHAP bar plot, features with longer bars are more influential overall. The summary dot plot provides insight into how each feature’s value (color) correlates with impact on the prediction (SHAP value on the x-axis). In the force plot for a single instance, features pushing the prediction higher (positive SHAP) are shown in red, and those pushing it lower are in blue, giving a clear explanation for that individual prediction.

 **Local Explanation with LIME**

In [ ]:
!pip install lime

import numpy as np
from lime.lime_tabular import LimeTabularExplainer

X_train_array = train[features].values
feature_names = list(features)
class_names = ['target']

explainer = LimeTabularExplainer(
    X_train_array,
    mode='regression',
    feature_names=feature_names,
    class_names=class_names,
    discretize_continuous=True
)

instance_idx = 0
instance = test[features].iloc[instance_idx].values

exp = explainer.explain_instance(instance, model_to_explain.predict, num_features=10)

print(f"LIME explanation for test instance {instance_idx}:")
for feature, contribution in exp.as_list():
    print(f"  {feature}: {contribution:.3f}")

In [ ]:
exp.show_in_notebook(show_table=True)

**Part 5**

Finally, we compile our insights and evaluate the overall ADS solution in terms of robustness, fairness, and practical implications.

**Model Performance and Validation**

In [ ]:
overall_cv_rmse = np.sqrt(mean_squared_error(train_results['actual'], train_results['prediction']))
print(f"Overall CV RMSE: {overall_cv_rmse:.4f}")

 **Quadratic Weighted Kappa (QWK)**

In [ ]:
from sklearn.metrics import cohen_kappa_score
actual_bins = pd.qcut(train_results['actual'], q=5, labels=False)
pred_bins = pd.qcut(train_results['prediction'], q=5, labels=False)

qwk = cohen_kappa_score(actual_bins, pred_bins, weights='quadratic')
print(f"Overall CV QWK: {qwk:.4f}")


**Robustness Analysis**

In [ ]:
group_metrics = train_results.groupby('spending_tier').apply(
    lambda df: np.sqrt(mean_squared_error(df['actual'], df['prediction']))
)
print(group_metrics)